In [23]:
import pandas as pd
import re

def parse_file(text_file):    
    pat = re.compile(r'^(\d\d\/\d\d\/\d\d.*?)(?=^^\d\d\/\d\d\/\d\d|\Z)', re.S | re.M)
    with open(text_file) as f:
        data = [m.group(1).strip().replace('\n', ' ') for m in pat.finditer(f.read())]
    #print(f'data {data}')
    sender = []; message = []; datetime = []
    for row in data:
        datetime.append(row.split(' - ')[0])
        try:
            s = re.search('m - (.*?):', row).group(1)
            sender.append(s)
        except:
            sender.append('')
        try:
            message.append(row.split(': ', 1)[1])
        except:
            message.append('')
    df = pd.DataFrame(zip(datetime, sender, message), columns=['timestamp', 'sender', 'message'])
    df['timestamp'] = pd.to_datetime(df.timestamp, format='%d/%m/%y, %I:%M %p')
    df = df[df.sender != ''].reset_index(drop=True)    
    return df

df = parse_file('chat.txt')

In [22]:
df.head()

,timestamp,sender,message
0,2018-11-15 20:32:00,Kanak Dahake Jr.,<Media omitted>
1,2018-11-15 20:47:00,P1,😆😆😆
2,2018-11-15 20:47:00,P1,kama nimitya
3,2018-11-15 20:47:00,P1,aaji kde
4,2018-11-15 20:50:00,Kanak Dahake Jr.,Sala 😂😂


In [21]:
sender = list(df.sender.unique())
sender
for s in sender:
    print(f'Message send by {s} is {df[df.sender==s].shape[0]}')

Message send by Kanak Dahake Jr. is 825
Message send by P1 is 1110
